# The Canonical Time-series Characteristics (catch22) transform

catch22\[1\] is a collection of 22 time series features extracted from the 7000+ present in the _hctsa_\[2\]\[3\] toolbox.
A hierarchical clustering was performed on the correlation matrix of features that performed better than random chance to remove redundancy.
These clusters were sorted by balanced accuracy using a decision tree classifier and a single feature was selected from the 22 clusters formed, taking into account balanced accuracy results, computational efficiency and interpretability.

In this notebook, we will demonstrate how to use the catch22 transform on the GunPoint univariate and BasicMotions multivariate datasets. We also show catch22 used for classication with a random forest classifier.

#### References:

\[1\] Lubba, C. H., Sethi, S. S., Knaute, P., Schultz, S. R., Fulcher, B. D., & Jones, N. S. (2019). catch22: CAnonical Time-series CHaracteristics. Data Mining and Knowledge Discovery, 33(6), 1821-1852.

\[2\] Fulcher, B. D., & Jones, N. S. (2017). hctsa: A computational framework for automated time-series phenotyping using massive feature extraction. Cell systems, 5(5), 527-531.

\[3\] Fulcher, B. D., Little, M. A., & Jones, N. S. (2013). Highly comparative time-series analysis: the empirical structure of time series and their methods. Journal of the Royal Society Interface, 10(83), 20130048.

## 1. Imports

In [1]:
from sktime.transformers.series_as_features.catch22_features import Catch22
from sktime.classification.hybrid._catch22_forest_classifier \
    import Catch22ForestClassifier

from sklearn import metrics
from sktime.datasets import load_gunpoint, load_basic_motions

## 2. Load data
For more details on the data set, see the [univariate time series classification notebook](https://github.com/alan-turing-institute/sktime/blob/master/examples/02_classification_univariate.ipynb).

In [2]:
GP_X_train, GP_y_train = load_gunpoint(split='train', return_X_y=True)
GP_X_test, GP_y_test = load_gunpoint(split='test', return_X_y=True)

print(GP_X_train.shape, GP_y_train.shape, GP_X_test.shape, GP_y_test.shape)

BM_X_train, BM_y_train = load_basic_motions(split='train', return_X_y=True)
BM_X_test, BM_y_test = load_basic_motions(split='test', return_X_y=True)

print(BM_X_train.shape, BM_y_train.shape, BM_X_test.shape, BM_y_test.shape)

(50, 1) (50,) (150, 1) (150,)
(40, 6) (40,) (40, 6) (40,)


## 3. catch22 transform

### Univariate

The catch22 features are provided in the form of a tranfromer, `Catch22`.
From this the transformed data can be used for a variety of time series analysis tasks.

In [17]:
c22_uv = Catch22()
c22_uv.fit(GP_X_train, GP_y_train)

Catch22()

In [15]:
transformed_data_uv = c22_uv.transform(GP_X_train)
print(transformed_data_uv.head())

         0         1   2   3         4         5         6     7       8   9   \
0 -0.519634 -0.651047  19  48  0.748442 -0.000027  0.221477  47.0  0.0400   0   
1 -0.540260 -0.677230  19  50  0.753535 -0.000062  0.248322  46.0  0.0800   0   
2 -0.534033 -0.658352  21  53  0.772093  0.000493  0.221477  50.0  0.0625   0   
3 -0.534749 -0.663529  21  53  0.791940  0.000194  0.241611  49.0  0.0400   0   
4 -0.454774 -0.594117  18  48  0.703249 -0.000209  0.214765  43.0  0.0400   0   

   ...        12        13        14        15    16        17     18     19  \
0  ...  0.642857  0.100000 -0.326667  0.989975  29.0  1.315707  0.150  0.850   
1  ...  0.703704  0.026667 -0.306667  0.990176  26.0  1.296189  0.775  0.825   
2  ...  1.033333  0.026667 -0.360000  0.989140  34.0  1.253208  0.775  0.850   
3  ...  0.965517 -0.020000  0.333333  0.989461  33.0  1.296189  0.750  0.850   
4  ...  0.576923 -0.060000  0.300000  0.989789  19.0  1.323159  0.200  0.850   

         20        21  
0  0.049

The transform `Catch22` method will process all 22 features.
For individual features, the transform_single_feature method can be used when provided with a numeric feature ID or the feature name.

In [16]:
transformed_feature_uv = c22_uv.transform_single_feature(GP_X_train, "CO_f1ecac")
print(transformed_feature_uv)

[19 19 21 21 18 20 19 19 21 23 23 22 22 22 18 23 21 19 22 20 22 22 22 16
 19 21 22 23 21 21 23 17 21 21 19 19 19 19 16 20 20 22 23 23 21 19 20 19
 19 17]


### Multivariate

Transformation of multivariate data is supported by `Catch22`.
The default procedure will concatenate each column prior to transformation.

In [11]:
c22_mv = Catch22()
c22_mv.fit(BM_X_train, BM_y_train)

Catch22()

In [12]:
transformed_data_mv = c22_mv.transform(BM_X_train)
print(transformed_data_mv.head())

         0         1   2   3         4         5         6     7         8   \
0 -0.417887  0.187460   2   5  0.105551 -0.649750  0.794658   9.0  0.002256   
1  1.061005  0.294088   2   6  0.043438 -1.968673  0.626043  79.0  0.012387   
2 -0.065333  0.297270   3   6  0.111391 -0.015163  0.808013  25.0  0.000808   
3  0.274125 -0.130168   2   5  0.062411 -0.103788  0.841402  17.0  0.005236   
4 -0.310012  0.068879   3   5  0.086561  0.321862  0.813022  26.0  0.012837   

   9   ...        12        13        14        15   16        17        18  \
0   9  ...  0.666667 -0.576667 -0.421667  0.479256  7.0  1.952422  0.127660   
1  10  ...  0.250000 -0.501667 -0.318333  0.603359  7.0  1.858507  0.170213   
2  11  ...  1.000000 -0.446667 -0.470000  0.812749  8.0  1.927062  0.127660   
3   8  ...  0.666667 -0.366667 -0.351667  0.486981  7.0  2.009502  0.127660   
4  10  ...  0.750000 -0.501667 -0.695833  0.767016  7.0  1.869634  0.787234   

         19        20        21  
0  0.659574  0.6

## 4. catch22 Forest Classifier

For classification tasks the default classifier to use with the catch22 features is random forest classifier.
An implementation making use of the `BaggingClassifier` from sklearn built on catch22 features is provided in the form on the `Catch22ForestClassifier` for ease of use.

In [18]:
c22f = Catch22ForestClassifier(n_estimators=100)
c22f.fit(GP_X_train, GP_y_train)

D:\UEAMachineLearning\sktime-workshop-boss\sktime\classification\hybrid\_catch22_forest_classifier.py:48: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  y = check_array(y, ensure_2d=False)


Catch22ForestClassifier(bootstrap=True, n_estimators=100, n_jobs=None,
                        random_state=None)

In [19]:
c22f_preds = c22f.predict(GP_X_test)
print("C22F Accuracy: " + str(metrics.accuracy_score(GP_y_test, c22f_preds)))

C22F Accuracy: 0.9133333333333333
